Load moduls

In [1]:
import numpy as np
import scipy.stats as stats
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

%matplotlib inline
%config InlineBackend.figure_format ='retina'

# Suburb

# House

In [2]:
df = pd.read_csv('data/houses.csv')

In [5]:
df.head(2)

,solddate,price,address1,postcode,latitude,longitude,bedrooms,bathrooms,carparkings,agentbrand,location,suburb
0,08-May-17,165000,907/268 Flinders Street,3000,-37.817764,144.965652,0,1,0,Inner Real Estate NEXTRE,"('-37.8177643', '144.965652')",MELBOURNE
1,11-Mar-17,187700,7/9-13 Anthony Street,3000,-37.809220,144.958939,0,1,0,Purplebricks VIC,"('-37.80922', '144.958939')",MELBOURNE


In [9]:
mask = (df.bedrooms == 2) | (df.bedrooms == 3)
df = df[mask]

In [17]:
df.postcode.nunique()

622

In [ ]:
df.to_csv('house.csv', index=False)